# **Torgo Speakers spelling correction training script using machine translation**

### **Objective: Spelling correction training for Torgo dataset speakers using machine translation**

### **Ensure that GPU and RAM is set up: will be needed for training purpose**

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
# ensure enough memory present so that training does not stop
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

### **Install the libraries**

In [ ]:
# Install required libraries
!pip install datasets
!pip install transformers==4.28.0
!pip install accelerate
!pip install jiwer
!pip install huggingface_hub

### **Import libraries**

In [ ]:
# Import libraries
import torch
from transformers import BartTokenizerFast, BartForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from jiwer import wer
from huggingface_hub import notebook_login

In [ ]:
# Login to Hugging Face
notebook_login()

### **Mount the json files from Google Drive**

In [ ]:
# mount other_speakers.json file is stored in Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Load the dataset from the JSON files
dataset = load_dataset('json', data_files={'train': '/content/drive/MyDrive/f01_other_speakers.json',
                                           'test': '/content/drive/MyDrive/speaker_f01.json'})

In [ ]:
print(dataset)

In [ ]:
print(dataset['train'][:5])
print(dataset['test'][:5])

By setting a fixed random seed, the data will be split into training and validation sets in a consistent manner each time the code is executed. This is useful for debugging, testing, and comparing different runs of the code. The choice of the number 42 as the seed is arbitrary and can be any integer value. The important aspect is to use the same seed consistently if reproducibility is desired.

In [ ]:
# Split the dataset into train and validation sets
train_dataset, val_dataset = train_test_split(dataset['train'], test_size=0.1, random_state=42)

In [ ]:
from datasets import Dataset

train_data = Dataset.from_dict(train_dataset)  # Convert the train data to a dataset
val_data = Dataset.from_dict(val_dataset)      # Convert the validation data to a dataset

In [ ]:
print(type(train_data))
print(type(val_data))

In [ ]:
# Check column names in train dataset
print(train_data.column_names)

# Check column names in validation dataset
print(val_data.column_names)

In [ ]:
print(train_data['actual'][:5])
print(train_data['prediction'][:5])
#print(val_data[:5])

In [ ]:
# Define the source and target language columns
source_lang = 'prediction'
target_lang = 'actual'

In [ ]:
print(source_lang)

In [ ]:
# Define the max_length for padding and truncation
max_length = 512

The preprocessing function serves to prepare the data for training or evaluation. It uses a tokenizer to tokenize the inputs and labels, formats the inputs by adding a source language identifier, encodes the tokenized inputs and labels, and creates a dictionary of model inputs. The function ensures that the data is properly tokenized, formatted, and encoded according to the model's requirements. It helps maintain consistency and compatibility between the input data and the model during training or evaluation.

In [ ]:
# Initialize the tokenizer
tokenizer = BartTokenizerFast.from_pretrained('facebook/bart-base')

# Tokenize the data
# The preprocess_function function is defined to preprocess the data by tokenizing the inputs and labels
def preprocess_function(examples):
    inputs = [f'{source_lang}: {text}' for text in examples[source_lang]]
    targets = examples[target_lang]
    encoding = tokenizer(inputs, padding=True, truncation=True, return_tensors='pt', max_length=max_length)
    model_inputs = {
        'input_ids': encoding['input_ids'].squeeze(),
        'attention_mask': encoding['attention_mask'].squeeze(),
        'labels': tokenizer(targets, padding=True, truncation=True, return_tensors='pt')['input_ids'].squeeze()
    }
    return model_inputs

In [ ]:
# Select a random data point from the train dataset
sample_data = train_data[0]

# Call the preprocess function on the sample data
processed_data = preprocess_function(sample_data)

# Inspect the output
print(processed_data)

In [ ]:
#train_data = preprocess_function(train_data)
#val_data = preprocess_function(val_data)

# Apply preprocess_function to train_data and val_data
train_data = train_data.map(preprocess_function, batched=True)
val_data = val_data.map(preprocess_function, batched=True)

In [ ]:
# Access a few samples from train_data
for i in range(5):
    sample_input_ids = train_data['input_ids'][i]
    sample_attention_mask = train_data['attention_mask'][i]
    sample_labels = train_data['labels'][i]

    print(f"Sample {i+1}:")
    print("Input IDs:", sample_input_ids)
    print("Attention Mask:", sample_attention_mask)
    print("Labels:", sample_labels)
    print()

A data loader is a component used in machine learning frameworks, such as PyTorch, to handle the loading and batching of data during the training or evaluation process. Its main purpose is to efficiently provide batches of data to the model for processing.

### **Train the model**

The purpose of a data collator is to take a list of samples from a dataset and collate them into a batch that can be processed by the model during training or evaluation. It works closely with the data loader to handle the specific requirements of the model's input format.

In [ ]:
# define a data_collator function for batch processing
def data_collator(features):
    batch = {}
    # Pad input_ids and attention_mask to the maximum length within the batch
    max_length = max(len(feature['input_ids']) for feature in features)
    batch['input_ids'] = torch.stack([torch.tensor(feature['input_ids'] + [tokenizer.pad_token_id] * (max_length - len(feature['input_ids']))) for feature in features])
    batch['attention_mask'] = torch.stack([torch.tensor(feature['attention_mask'] + [0] * (max_length - len(feature['attention_mask']))) for feature in features])
    batch['labels'] = torch.stack([torch.tensor(feature['labels'] + [-100] * (max_length - len(feature['labels']))) for feature in features])
    return batch

The data loader is responsible for loading and batching the data, while the data collator is responsible for formatting and aligning the data within each batch. They serve different functions in the training process.

In [ ]:
# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="spell_correction_F01",
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    push_to_hub=True,
)

In [ ]:
# verify passing the correct inputs to the trainer
print("Train Dataset:", train_data)
print("Validation Dataset:", val_data)
print("Tokenizer:", tokenizer)
print("Training Arguments:", training_args)

In [ ]:
# model is initialized with the BARTForConditionalGeneration class and moved to the GPU if available.
model = BartForConditionalGeneration.from_pretrained('facebook/bart-base')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
# DO NOT RUN THE CODE BLOCK
# code to keep google collab running:
# Right click -> Inspect -> Console -> paste the code below
function ClickConnect(){
  console.log("Clicking on the page");
  document.querySelector("colab-connect-button").click()
}
setInterval(ClickConnect, 60000) // Clicks every 1 minute

In [ ]:
# The Seq2SeqTrainer is created with the defined model, training arguments, datasets, tokenizer, and data_collator
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    data_collator=data_collator
)

trainer.train()